# Modeling

In [81]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

## Load In Data

### Y Data

In [82]:
#X Data
with open('X.pkl','rb') as f:
    X = pickle.load(f)
    print(len(X))
    
#Y Data
with open('Y.pkl','rb') as f:
    Y = pickle.load(f)
    print(len(Y))


51
51


In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [84]:
X_train[0]

array([[-8.9657059e+00, -8.9464569e+00, -9.0931015e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-4.2739410e+00, -4.2424545e+00, -4.2462006e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 2.7397156e-02,  3.3378601e-02, -5.4862976e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-1.2730175e-02, -5.8371192e-01,  5.4546839e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.3081874e-01, -7.4142410e-04, -5.0024819e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.5824046e-02,  8.4550835e-02, -8.5358843e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [85]:
X_train_scaled, X_test_scaled = Scaler(X_train, X_test)

In [86]:
def Scaler(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = []
    X_test_scaled = []
    for i in range(38):
        scaler.fit(X_train[i])
        X_train_scaled.append(scaler.transform(X_train[i]))
    for i in range(13):
        X_test_scaled.append(scaler.transform(X_test[i]))
    return np.array(X_train_scaled), np.array(X_test_scaled)

In [87]:
X_train_scaled, X_test_scaled = Scaler(X_train, X_test)
print("X Train Shape: ", X_train_scaled.shape)
print("X Test Shape: ", X_test_scaled.shape)
print("Number of features: ", len(X[0]))
print("Number of observations: ", len(X[0][0]))

X Train Shape:  (38, 12, 32409)
X Test Shape:  (13, 12, 32409)
Number of features:  12
Number of observations:  32409


# Construct a Baseline Model

In [88]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(12, 32409)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=3,verbose=2)

In [ ]:
tf.keras.layers.InputLayer(
    input_shape=None,
    batch_size=None,
    dtype=None,
    input_tensor=None,
    sparse=None,
    name=None,
    ragged=None,
    type_spec=None,
    **kwargs
)